# HexPosition and GosperCurve

> Cube coordinate system for hexagonal grids

In [ ]:
#| default_exp plot.cube

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import numpy as np
import math
from functools import reduce
from dataclasses import dataclass, field
from fastcore.basics import patch
from HexMagic.styles import StyleCSS

In [ ]:
#| export
from HexMagic.plot.primitives import MapCord, MapPath , PrimitiveDemo

In [ ]:
#| export
@dataclass
class HexPosition:
    _coords: np.ndarray = field(default_factory=lambda: np.array([0, 0, 0]))
    label: str = ""

    _theDirs = {(1, 0, -1):"E",
        (1, -1, 0):"NE",
        (0, -1, 1):"NW",
        (-1, 0, 1):"W",
        (-1, 1, 0):"SW", 
        (0, 1, -1):"SE",
        (0, 0, 0):"X"
        }

        # Add reverse mapping
    _labelToDirs = {v: k for k, v in _theDirs.items()}
    _rings = []
    _neigbors = []
    
    def __init__(self, q: int = 0, r: int = 0, s: int = 0, label: str = ""):
        self._coords = np.array([q, r, s])
        self.label = label
    
    @property
    def q(self) -> int: return int(self._coords[0])
    
    @property
    def r(self) -> int: return int(self._coords[1])
    
    @property
    def s(self) -> int: return int(self._coords[2])

    @classmethod 
    def direction(cls, lbl):
        coords = HexPosition._labelToDirs.get(lbl, (0, 0, 0))
        return cls(*coords, label=lbl)
     
    def desc(self):
        return HexPosition._theDirs.get((self.q,self.r,self.s),  self.label)

    @classmethod
    def origin(cls)->'HexPosition':
        return HexPosition(0,0,0,"origin")
   
    @classmethod
    def directions(cls):

        return HexPosition._orderDirs 

    def copy(self,label:None):
        if label is None:
            label = self.label
        return HexPosition(self.q,self.r,self.s,label)

    @property
    def cube_coords(self):
        return (self.q, self.r, self.s)
    
    def encode(self) -> str:
        """Encode as string like '2^3' (ring^position)"""
        return f"{self.q}^{self.r}^{self.s}"

   
    
    def pixel(self, hex_radius: float, center: MapCord) -> tuple[float, float]:
        """Convert HexPosition cube coordinates to pixel position for pointy-top hex."""
        x = hex_radius * (math.sqrt(3) * self.q + math.sqrt(3)/2 * self.r)
        y = hex_radius * (3/2 * self.r)
        return MapCord(center.x + x, center.y + y)
    
    @staticmethod
    def decode(s: str) -> 'HexPosition':
        """Decode from string like '2^3'"""
        vals = s.split('^')
        return HexPosition(q=int(vals[0]), r=int(vals[1]),s=int(vals[2]))

    @staticmethod
    def _cube_round(q: float, r: float, s: float) -> 'HexPosition':
        """Round fractional cube coords to nearest valid hex."""
        rq, rr, rs = round(q), round(r), round(s)
        dq, dr, ds = abs(rq - q), abs(rr - r), abs(rs - s)
        
        # Adjust the largest diff to maintain q + r + s = 0
        if dq > dr and dq > ds:
            rq = -rr - rs
        elif dr > ds:
            rr = -rq - rs
        else:
            rs = -rq - rr
        
        return HexPosition(int(rq), int(rr), int(rs))

    @staticmethod
    def average(positions: list['HexPosition']) -> 'HexPosition':
        """Calculate the average position of a list of HexPositions."""
        if not positions:
            return HexPosition.origin()
        
        total = sum(positions, HexPosition.origin())
        return HexPosition._cube_round(total.q / len(positions), 
                                        total.r / len(positions), 
                                        total.s / len(positions))

    @staticmethod
    def weighted_average(positions: list['HexPosition'], weights: list[float]) -> 'HexPosition':
        """Calculate weighted average of HexPositions.
        
        Args:
            positions: List of HexPositions
            weights: List of weights (same length as positions)
        """
        if not positions or not weights:
            return HexPosition.origin()
        
        if len(positions) != len(weights):
            raise ValueError("positions and weights must have same length")
        

        weighted_sum = reduce(lambda acc, pw: acc + pw[1] * pw[0], 
                            zip(positions, weights), 
                            HexPosition.origin())
        
        total_weight = sum(weights)
        if total_weight == 0:
            return HexPosition.origin()
        
        return HexPosition._cube_round(weighted_sum.q / total_weight,
                                        weighted_sum.r / total_weight,
                                        weighted_sum.s / total_weight)


HexPosition._orderDirs = [HexPosition.direction(lbl) for lbl in ["SW","W","NW", "NE", "E", "SE"]]


In [ ]:
#| export
@patch
def distance(self: HexPosition, other: HexPosition) -> int:
    """Cube coordinate distance using numpy"""
    return int(np.abs(self._coords - other._coords).sum() // 2)

@patch
def __add__(self: HexPosition, other: 'HexPosition') -> 'HexPosition':
    return HexPosition(*(self._coords + other._coords))

@patch
def __sub__(self: HexPosition, other: 'HexPosition') -> 'HexPosition':
    return HexPosition(*(self._coords - other._coords))

@patch
def __rmul__(self: HexPosition, scalar: int) -> 'HexPosition':
    return HexPosition(*(self._coords * scalar))

@patch
def __neg__(self: HexPosition) -> 'HexPosition':
    return HexPosition(*(-self._coords))

@patch
def __abs__(self: HexPosition) -> int:
    """Distance from origin: abs(pos)"""
    return self.distance(HexPosition.origin())

@patch
def __eq__(self: HexPosition, other) -> bool:
    """Equality: pos1 == pos2"""
    if not isinstance(other, HexPosition):
        return False
    return self.q == other.q and self.r == other.r and self.s == other.s


@patch
def __hash__(self: HexPosition) -> int:
    """Hash for use in sets/dicts"""
    return hash((self.q, self.r,self.s))

In [ ]:
#| export
@patch
def __repr__(self: HexPosition) -> str:
    """Better string representation"""
    if self.label:
        return f"HexPosition({self.q}, {self.r}, {self.s}, '{self.label}')"
    return f"HexPosition({self.q}, {self.r}, {self.s})"

## rotations and rings

In [ ]:
#| export
@patch
def rotate_left(self: HexPosition) -> 'HexPosition':
    """Rotate 60° counter-clockwise: (q,r,s) -> (-s,-q,-r)"""
    return HexPosition(*(-np.roll(self._coords, 1)))

@patch
def rotate_right(self: HexPosition) -> 'HexPosition':
    """Rotate 60° clockwise: (q,r,s) -> (-r,-s,-q)"""
    return HexPosition(*(-np.roll(self._coords, -1)))

@patch
def rotate(self: HexPosition, steps: int = 1) -> HexPosition:
    """Rotate around origin. Positive = counter-clockwise, negative = clockwise."""
    steps = steps % 6
    coords = self._coords
    # Apply sign pattern: even rotations keep sign, odd rotations negate
    sign = 1 if steps % 2 == 0 else -1
    rolled = np.roll(coords, steps)
    return HexPosition(*(sign * rolled), label=self.label)

In [ ]:
#| export
@patch
def _ring(self: HexPosition, radius: int, clockwise: bool = True) -> list['HexPosition']:
    """Generate all HexPositions in a ring at given radius from center."""
    if radius == 0:
        return [self]
    
    results = []
    current = self + radius * HexPosition.direction("E")
    
    # SW for counter-clockwise, SE for clockwise
    direction = HexPosition.direction("E").rotate(2 if not clockwise else -2)
    rotation = 1 if not clockwise else -1
    
    for side in range(6):
        for _ in range(radius):
            results.append(current)
            current = current + direction
        direction = direction.rotate(rotation)
    
    return results

In [ ]:
#| export
@patch
def _neighbor(self: HexPosition, direction: int) -> 'HexPosition':
    """Get neighbor in direction (0-5, starting East and going counter-clockwise)"""
    directions = HexPosition.directions()
    return self + directions[direction % 6]

HexPosition._rings = [HexPosition.origin()._ring(i) for i in range(10)]
HexPosition._neighbors = [HexPosition.origin()._neighbor(i) for i in range(6)]


In [ ]:
#| export
@patch
def ring(self: HexPosition, radius: int, clockwise: bool = True) -> list['HexPosition']:
    """Generate all HexPositions in a ring at given radius from center.
    
    Uses cached rings for radius < 10 when clockwise and centered at origin.
    """
    # Fast path: use cached rings if at origin, clockwise, and radius < 10
    if radius < 10 and clockwise and self.q == 0 and self.r == 0 and self.s == 0:
        return HexPosition._rings[radius]
    
    # Fast path: use cached rings and offset if radius < 10 and clockwise
    if radius < 10 and clockwise:
        return [self + pos for pos in HexPosition._rings[radius]]
    
    # Slow path: compute on demand
    return self._ring(radius, clockwise)

In [ ]:
#| export
@patch
def neighbor(self: HexPosition, direction: int) -> 'HexPosition':
    """Get neighbor in direction (0-5, starting East going counter-clockwise).
    
    Uses cached neighbor offsets for better performance.
    """
    # Use cached neighbors
    return self + HexPosition._neighbors[direction % 6]

## Movement

In [ ]:
#| export
@patch
def line_to(self: HexPosition, other: HexPosition) -> list[HexPosition]:
    """Get line of hexes from self to other using linear interpolation"""
    n = self.distance(other)
    if n == 0:
        return [self]

    results = []
    for i in range(n + 1):
        t = i / n
        # Lerp in cube coordinates using numpy
        coords = self._coords + (other._coords - self._coords) * t
        results.append(HexPosition._cube_round(*coords))

    return results

In [ ]:
#| export
@patch
def path_through_waypoints(self: HexPosition, waypoints: list[HexPosition], 
                           destination: HexPosition) -> list[HexPosition]:
    """Create a path from self through all waypoints to destination.
    
    Args:
        waypoints: List of intermediate points to visit
        destination: Final destination
        use_lines: If True, use straight lines. If False, use radial paths (default)
    """
    if not waypoints:
        return self.line_to(destination) 
    
    full_path = []
    current = self
    path_method = (lambda a, b: a.line_to(b)) 
    
    # Go through each waypoint
    for waypoint in waypoints:
        segment = path_method(current, waypoint)
        # Avoid duplicates at segment boundaries
        if full_path and segment and full_path[-1].cube_coords == segment[0].cube_coords:
            full_path.extend(segment[1:])
        else:
            full_path.extend(segment)
        current = waypoint
    
    # Final segment to destination
    final_segment = path_method(current, destination)
    if full_path and final_segment and full_path[-1].cube_coords == final_segment[0].cube_coords:
        full_path.extend(final_segment[1:])
    else:
        full_path.extend(final_segment)
    
    return full_path

In [ ]:
#| export
@patch
def move_in(self: HexPosition, center: HexPosition) -> HexPosition:
    """Move one step toward center."""
    if self == center:
        return self
    return self.line_to(center)[1]

@patch
def move_out(self: HexPosition, center: HexPosition) -> HexPosition:
    """Move one step away from center."""
    if self == center:
        return self
    return 2 * self - self.move_in(center)


# Set direction attributes
for x in ["E","NE","NW", "W","SW", "SE"]:
    setattr(HexPosition, x, HexPosition.direction(x))

In [ ]:
east = HexPosition.E
east

In [ ]:
#| export
@patch
def postionDemo(self: PrimitiveDemo):
    east = HexPosition.E
    print(f"this is east {HexPosition.E}")
    middle = HexPosition.origin()
    print(f"this is middle {middle}")
    ring = middle.ring(8)[:5]
    print(f"this is the first few in the ring 8 away frome east {ring}")
    
    path = middle.line_to(HexPosition(6, 3, -9))
    print(f"making a line {path}")

    print("This is how we would walk from one to another")
    start = middle
    for pos in path:
        print((pos - start).desc())
        start = pos

    lineMiddle = HexPosition.average(path)
    print(f"the middle of the line is {lineMiddle}")

In [ ]:
PrimitiveDemo().postionDemo()

### Helpers

I am looking for functions that do an average and weighted average of a list of hexposition.
for average could I just do total_pos = sum([theList]) ret = HexPositin cuberound HexPositin * 1/len(theList) and something similar maybe using the map/filter/reduce for the weighted average

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're on the right track! For the average, your approach is close. Here's what would work:

```python
total = sum(positions, HexPosition.origin())
avg = HexPosition._cube_round(total.q / len(positions), 
                                total.r / len(positions), 
                                total.s / len(positions))
```

You need to divide each coordinate individually before rounding, since `HexPosition` doesn't support scalar division directly.

For weighted average, yes! You could use something like:

```python
from functools import reduce

weighted_sum = reduce(lambda acc, pw: acc + pw[1] * pw[0], 
                      zip(positions, weights), 
                      HexPosition.origin())
```

Then divide and round the same way.

Would you like me to show you complete functions for both, or would you prefer to try implementing them yourself first?

can you write them?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
@patch
def average(positions: list[HexPosition]) -> HexPosition:
    """Calculate the average position of a list of HexPositions."""
    if not positions:
        return HexPosition.origin()
    
    total = sum(positions, HexPosition.origin())
    return HexPosition._cube_round(total.q / len(positions), 
                                    total.r / len(positions), 
                                    total.s / len(positions))

@patch
def weighted_average(positions: list[HexPosition], weights: list[float]) -> HexPosition:
    """Calculate weighted average of HexPositions.
    
    Args:
        positions: List of HexPositions
        weights: List of weights (same length as positions)
    """
    if not positions or not weights:
        return HexPosition.origin()
    
    if len(positions) != len(weights):
        raise ValueError("positions and weights must have same length")
    
    from functools import reduce
    
    weighted_sum = reduce(lambda acc, pw: acc + pw[1] * pw[0], 
                         zip(positions, weights), 
                         HexPosition.origin())
    
    total_weight = sum(weights)
    if total_weight == 0:
        return HexPosition.origin()
    
    return HexPosition._cube_round(weighted_sum.q / total_weight,
                                    weighted_sum.r / total_weight,
                                    weighted_sum.s / total_weight)
```

Note: These are class methods (using `@patch`), so you'd call them like `HexPosition.average([pos1, pos2, pos3])` and `HexPosition.weighted_average([pos1, pos2], [0.3, 0.7])`.

code example

I think we can do some better versions of these hexPostion functions such as hex_dot

it would be easier to use the underlying numpy
```
@patch
def _boundary_type(self: Geology, plate_a: 'Plate', plate_b: 'Plate') -> str:
    """Determine boundary type between two plates using HexPosition directions.
    
    Uses cube coordinate math:
    - Convergent: plates moving toward each other
    - Divergent: plates moving apart
    - Transform: plates sliding past each other
    """
    # Get plate directions (HexPosition or None)
    dir_a = getattr(plate_a, 'direction', None)
    dir_b = getattr(plate_b, 'direction', None)
    
    # If no directions, use plate types as heuristic
    if dir_a is None or dir_b is None:
        # Oceanic + Continental = usually convergent (subduction)
        if plate_a.kind != plate_b.kind:
            return 'convergent'
        # Two oceanic = often divergent (spreading ridge)
        elif plate_a.kind == PlateKind.oceanic:
            return 'divergent'
        # Two continental = mixed, default to transform
        else:
            return 'transform'
    
    # Get plate centers as HexPositions
    center_a_idx = plate_a.region.centroid_hex()
    center_b_idx = plate_b.region.centroid_hex()
    
    center_a_pos = self.terrain.hexGrid.index_to_hexposition(center_a_idx)
    center_b_pos = self.terrain.hexGrid.index_to_hexposition(center_b_idx)
    
    # Vector from A to B in cube coordinates
    to_b = HexPosition(
        center_b_pos.q - center_a_pos.q,
        center_b_pos.r - center_a_pos.r,
        center_b_pos.s - center_a_pos.s
    )
    
    # Normalize to unit direction (find closest of 6 directions)
    if to_b.q == 0 and to_b.r == 0 and to_b.s == 0:
        return 'transform'  # Same center, shouldn't happen
    
    # Dot product in cube coordinates: sum of component products
    # For HexPosition, we can use: dot = a.q*b.q + a.r*b.r + a.s*b.s
    def hex_dot(a: HexPosition, b: HexPosition) -> float:
        return a.q * b.q + a.r * b.r + a.s * b.s
    
    # Magnitude squared for normalization
    def hex_mag_sq(h: HexPosition) -> float:
        return h.q**2 + h.r**2 + h.s**2
    
    to_b_mag = math.sqrt(hex_mag_sq(to_b))
    if to_b_mag < 0.001:
        return 'transform'
    
    # Normalize dir_a and dir_b (they should already be unit-ish)
    dir_a_mag = math.sqrt(hex_mag_sq(dir_a))
    dir_b_mag = math.sqrt(hex_mag_sq(dir_b))
    
    if dir_a_mag < 0.001 or dir_b_mag < 0.001:
        return 'transform'
    
    # How much is A moving toward B?
    # Positive = toward, negative = away
    a_toward_b = hex_dot(dir_a, to_b) / (dir_a_mag * to_b_mag)
    
    # How much is B moving toward A? (opposite direction)
    b_toward_a = -hex_dot(dir_b, to_b) / (dir_b_mag * to_b_mag)
    
    # Combined approach metric
    approach = a_toward_b + b_toward_a
    
    # Check for transform: both moving perpendicular to the line between them
    # In hex space, perpendicular means low dot product with to_b
    a_parallel = abs(a_toward_b)
    b_parallel = abs(b_toward_a)
    
    # Classify
    if approach > 0.8:
        return 'convergent'
    elif approach < -0.8:
        return 'divergent'
    elif a_parallel < 0.4 and b_parallel < 0.4:
        # Both moving mostly perpendicular = transform
        return 'transform'
    else:
        # Ambiguous - use plate types
        if plate_a.kind != plate_b.kind:
            return 'convergent'
        return 'transform'
```

In [ ]:
@patch
def dot(self: HexPosition, other: HexPosition) -> float:
    """Dot product in cube coordinates."""
    return float(np.dot(self._coords, other._coords))

@patch
def mag_sq(self: HexPosition) -> float:
    """Magnitude squared (sum of squares)."""
    return float(np.dot(self._coords, self._coords))

@patch
def mag(self: HexPosition) -> float:
    """Magnitude (Euclidean length in cube space)."""
    return float(np.sqrt(self.mag_sq()))

@patch
def normalized(self: HexPosition) -> HexPosition:
    """Return unit vector in same direction (fractional coords, not rounded)."""
    m = self.mag()
    if m < 0.001:
        return HexPosition.origin()
    return HexPosition(*(self._coords / m))

I think we can do some better versions of these hexPostion functions such as hex_dot

it would be easier to use the underlying numpy

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Absolutely! Since `HexPosition` already stores `_coords` as a numpy array, these become one-liners:

```python
@patch
def dot(self: HexPosition, other: HexPosition) -> float:
    """Dot product in cube coordinates."""
    return float(np.dot(self._coords, other._coords))

@patch
def mag_sq(self: HexPosition) -> float:
    """Magnitude squared (sum of squares)."""
    return float(np.dot(self._coords, self._coords))

@patch
def mag(self: HexPosition) -> float:
    """Magnitude (Euclidean length in cube space)."""
    return float(np.sqrt(self.mag_sq()))

@patch
def normalized(self: HexPosition) -> HexPosition:
    """Return unit vector in same direction (fractional coords, not rounded)."""
    m = self.mag()
    if m < 0.001:
        return HexPosition.origin()
    return HexPosition(*(self._coords / m))
```

Then your boundary code becomes much cleaner:

```python
to_b_mag = to_b.mag()
a_toward_b = dir_a.dot(to_b) / (dir_a.mag() * to_b_mag)
b_toward_a = -dir_b.dot(to_b) / (dir_b.mag() * to_b_mag)
```

You might also want a convenience method for the normalized dot (cosine similarity):

```python
@patch
def cos_angle(self: HexPosition, other: HexPosition) -> float:
    """Cosine of angle between two vectors (normalized dot product)."""
    m1, m2 = self.mag(), other.mag()
    if m1 < 0.001 or m2 < 0.001:
        return 0.0
    return self.dot(other) / (m1 * m2)
```

### Play

In [ ]:
#| export
@patch
def spiral(self: HexPosition, radius: int,clockwise=True) -> list[HexPosition]:
    """Get all hexes in a true spiral pattern from center out to radius.
    
    Returns hexes in order where each hex is adjacent to the previous one,
    creating a continuous spiral path rather than concentric rings.
    """
    if radius == 0:
        return [self]
    
    results = [self]  # Start at center
    current = self
    
    # Start moving east
    last_direction = HexPosition.direction("E")
    
    for ring in range(1, radius + 1):
        # Move out one step to start the new ring
        current = current + last_direction
        results.append(current)
        
        # Walk around this ring - 6 sides, 'ring' steps per side
        for side in range(6):
            # Rotate counter-clockwise to next direction
            if clockwise:
                last_direction = last_direction.rotate_right()
            else:
                last_direction = last_direction.rotate_left()
            
            # Walk 'ring' steps in this direction
            # (but skip the first step on first side since we already moved out)
            steps = ring if side > 0 else ring - 1
            for step in range(steps):
                current = current + last_direction
                results.append(current)
    
    return results

In [ ]:
#| export
class GosperCurve:
    """Generate Gosper curves using L-system rules and cube coordinates."""
    
    def __init__(self, order: int = 3):
        self.order = order
        
    @staticmethod
    def _expand_lsystem(axiom: str, rules: dict, iterations: int) -> str:
        """Expand an L-system string."""
        current = axiom
        for _ in range(iterations):
            current = ''.join(rules.get(c, c) for c in current)
        return current
    
    @classmethod
    def generate_path(cls, order: int, start: HexPosition = None) -> list[HexPosition]:
        """
        Generate a Gosper curve path using L-system.
        
        The Gosper curve L-system:
        - Axiom: A
        - Rules: A -> A-B--B+A++AA+B-
                 B -> +A-BB--B-A++A+B
        - +: turn left 60° (counter-clockwise)
        - -: turn right 60° (clockwise)
        """
        start = start or HexPosition.origin()
        
        expanded = cls._expand_lsystem(
            axiom="A",
            rules={'A': "A-B--B+A++AA+B-", 'B': "+A-BB--B-A++A+B"},
            iterations=order
        )
        
        path = [start]
        current_pos = start
        current_dir = HexPosition.direction("E")
        
        for symbol in expanded:
            if symbol in ('A', 'B'):
                current_pos = current_pos + current_dir
                path.append(current_pos)
            elif symbol == '+':
                current_dir = current_dir.rotate(1)
            elif symbol == '-':
                current_dir = current_dir.rotate(-1)
        
        return path
    
    @classmethod
    def generate_terdragon(cls, order: int, start: HexPosition = None) -> list[HexPosition]:
        """
        Generate a Terdragon curve (triangular variant).
        
        L-system:
        - Axiom: F
        - Rules: F -> F+F-F
        - +: turn left 120° (2 hex steps)
        - -: turn right 120°
        """
        start = start or HexPosition.origin()
        
        expanded = cls._expand_lsystem(
            axiom="F",
            rules={'F': "F+F-F"},
            iterations=order
        )
        
        path = [start]
        current_pos = start
        current_dir = HexPosition.direction("E")
        
        for symbol in expanded:
            if symbol == 'F':
                current_pos = current_pos + current_dir
                path.append(current_pos)
            elif symbol == '+':
                current_dir = current_dir.rotate(2)
            elif symbol == '-':
                current_dir = current_dir.rotate(-2)
        
        return path
    
    @classmethod
    def generate_peano_hex(cls, order: int, start: HexPosition = None) -> list[HexPosition]:
        """
        Generate a hexagonal Peano curve (space-filling).
        
        L-system:
        - Axiom: XF
        - Rules: X -> XF+YF+XF-F-YF-F-XF+YF-X
                 Y -> YF-XF-YF+F+XF+F+YF-XF+Y
        """
        start = start or HexPosition.origin()
        
        expanded = cls._expand_lsystem(
            axiom="XF",
            rules={
                'X': "XF+YF+XF-F-YF-F-XF+YF-X",
                'Y': "YF-XF-YF+F+XF+F+YF-XF+Y"
            },
            iterations=order
        )
        
        path = [start]
        current_pos = start
        current_dir = HexPosition.direction("E")
        
        for symbol in expanded:
            if symbol == 'F':
                current_pos = current_pos + current_dir
                path.append(current_pos)
            elif symbol == '+':
                current_dir = current_dir.rotate(1)
            elif symbol == '-':
                current_dir = current_dir.rotate(-1)
        
        return path
    
    @staticmethod
    def calculate_bounds(hex_path: list[HexPosition]) -> tuple[int, int, int, int]:
        """Calculate bounding box: (min_q, max_q, min_r, max_r)"""
        if not hex_path:
            return (0, 0, 0, 0)
        
        qs = [h.q for h in hex_path]
        rs = [h.r for h in hex_path]
        return (min(qs), max(qs), min(rs), max(rs))
    
    @staticmethod
    def center_path(hex_path: list[HexPosition]) -> list[HexPosition]:
        """Center a path around the origin."""
        if not hex_path:
            return []
        
        min_q, max_q, min_r, max_r = GosperCurve.calculate_bounds(hex_path)
        center_q = (min_q + max_q) // 2
        center_r = (min_r + max_r) // 2
        offset = HexPosition(center_q, center_r, -center_q - center_r)
        
        return [pos - offset for pos in hex_path]
    
    @staticmethod
    def path_to_mappath(hex_path: list[HexPosition], 
                        hex_radius: float = 20.0,
                        center: MapCord = None,
                        style: StyleCSS = None) -> MapPath:
        """Convert a HexPosition path to a MapPath for rendering."""
        center = center or MapCord(400, 400)
        style = style or StyleCSS("gosper", stroke="#2196F3", stroke_width=2, fill="none")
        
        pixel_points = [hexpos.pixel(hex_radius, center) for hexpos in hex_path]
        return MapPath(pixel_points, style)

### Silly explanation

### The Hive: Understanding Hex Cube Coordinates

## A Tale of the Worker Bee and Her Queen

Deep within the hexagonal chambers of the hive, the Queen sits at the very center of her domain. Each day, worker bees venture out into the honeycomb, visiting cells in precise patterns, always knowing exactly where they are relative to their Queen. This is the story of how they navigate their hexagonal world using **cube coordinates**.

---

## Chapter 1: The Queen's Chamber (The Origin)

The Queen resides at the center of the hive, and her position is the reference point for everything:

```python
from primitives import HexPosition

# The Queen sits at the origin
queen = HexPosition(q=0, r=0, s=0)
print(f"The Queen's position: {queen}")
# Output: HexPosition(0, 0, 0)
```

### The Sacred Constraint

In the hexagonal world, there is a fundamental law that all bees must obey:

**q + r + s = 0**

This means the Queen's position (0, 0, 0) automatically satisfies this rule. It's not just a mathematical curiosity—it's what makes cube coordinates work for hexagons!

Think of it this way: imagine three axes extending from the Queen's chamber:
- The **q-axis** points northeast
- The **r-axis** points south  
- The **s-axis** points northwest

Because hexagons have 6 sides (not 4 like squares), we need 3 axes to describe them, but they're constrained so that if you know any two coordinates, you can always calculate the third.

---

## Chapter 2: The Six Directions (Cardinal Paths)

A worker bee named Beatrice needs to visit the six cells immediately surrounding the Queen. These are the **cardinal directions** of the hive:

```python
# Get all six neighboring positions
directions = HexPosition.directions()

for direction in directions:
    print(f"{direction.label:2} → q={direction.q:2}, r={direction.r:2}, s={direction.s:2}")
```

Output:
```
E  → q= 1, r= 0, s=-1
NE → q= 1, r=-1, s= 0
NW → q= 0, r=-1, s= 1
W  → q=-1, r= 0, s= 1
SW → q=-1, r= 1, s= 0
SE → q= 0, r= 1, s=-1
```

Notice how each direction still obeys the rule: **q + r + s = 0**

Let's verify:
- East (E): 1 + 0 + (-1) = 0 ✓
- Northeast (NE): 1 + (-1) + 0 = 0 ✓
- Northwest (NW): 0 + (-1) + 1 = 0 ✓

### Moving in Directions

Beatrice starts at the Queen's chamber and moves East:

```python
# Start at Queen's position
beatrice = HexPosition(0, 0, 0)

# Move East
directions = HexPosition.directions()
east = directions[0]  # First direction is East

# Add the direction to current position
new_position = beatrice + east
print(f"After moving East: {new_position}")
# Output: HexPosition(1, 0, -1)
```

The `__add__` method in HexPosition makes this elegant—Beatrice just adds the direction vector to her position!

---

## Chapter 3: The Rings (Concentric Chambers)

The hive is organized in concentric rings around the Queen. All cells in a ring are the same distance from the center.

### Ring 0: The Queen's Chamber

```python
queen = HexPosition(0, 0, 0)
ring_0 = queen.ring(0)
print(f"Ring 0 contains {len(ring_0)} cell: {ring_0[0]}")
# Output: Ring 0 contains 1 cell: HexPosition(0, 0, 0)
```

### Ring 1: The Queen's Guard (6 cells)

```python
ring_1 = queen.ring(1)
print(f"Ring 1 contains {len(ring_1)} cells:")
for pos in ring_1:
    print(f"  {pos}")
```

Output:
```
Ring 1 contains 6 cells:
  HexPosition(1, 0, -1)
  HexPosition(1, -1, 0)
  HexPosition(0, -1, 1)
  HexPosition(-1, 0, 1)
  HexPosition(-1, 1, 0)
  HexPosition(0, 1, -1)
```

### Ring 2: The Outer Circle (12 cells)

```python
ring_2 = queen.ring(2)
print(f"Ring 2 contains {len(ring_2)} cells")
# Output: Ring 2 contains 12 cells
```

**Pattern Discovery**: Each ring has 6 × radius cells (except ring 0):
- Ring 0: 1 cell (special case)
- Ring 1: 6 cells
- Ring 2: 12 cells
- Ring 3: 18 cells
- Ring n: 6n cells

---

## Chapter 4: Distance (How Far Has Beatrice Traveled?)

Beatrice has an important insight to share with the Queen, but she's currently gathering information from a distant cell. How far away is she?

```python
# Beatrice is at this position
beatrice = HexPosition(q=3, r=-1, s=-2)

# The Queen is at origin
queen = HexPosition(0, 0, 0)

# Calculate distance
distance = beatrice.distance(queen)
print(f"Beatrice is {distance} cells away from the Queen")
# Output: Beatrice is 3 cells away from the Queen
```

### The Distance Formula

The distance formula uses the cube coordinates cleverly:

```python
def distance(self: HexPosition, other: HexPosition) -> int:
    return (abs(self.q - other.q) + 
            abs(self.r - other.r) + 
            abs(self.s - other.s)) // 2
```

**Why divide by 2?** Because in cube coordinates, we're measuring along three axes, but hexagonal distance only needs the longest path through the three axes. The formula essentially finds the maximum coordinate difference.

Let's trace Beatrice's example:
- Δq = |3 - 0| = 3
- Δr = |-1 - 0| = 1  
- Δs = |-2 - 0| = 2
- Distance = (3 + 1 + 2) / 2 = 3 ✓

---

## Chapter 5: Finding the Path Home (Radial Navigation)

Beatrice has her insight! Now she must return to the Queen, but she wants to visit one cell per ring, getting closer with each step.

```python
# Beatrice's current position
beatrice = HexPosition(3, -1, -2)
queen = HexPosition(0, 0, 0)

# Find the path home, one ring at a time
path = beatrice.radial_path_to(queen)

print(f"Beatrice's journey home ({len(path)} steps):")
for i, pos in enumerate(path):
    dist = pos.distance(queen)
    print(f"  Step {i}: {pos} (distance: {dist})")
```

Output:
```
Beatrice's journey home (4 steps):
  Step 0: HexPosition(3, -1, -2) (distance: 3)
  Step 1: HexPosition(2, 0, -2) (distance: 2)
  Step 2: HexPosition(1, 0, -1) (distance: 1)
  Step 3: HexPosition(0, 0, 0) (distance: 0)
```

Each step brings Beatrice exactly one ring closer to the Queen!

### How It Works

The `radial_path_to` method:
1. Determines which ring Beatrice is on (distance from Queen)
2. For each inner ring, finds the closest cell in that ring
3. Steps through each ring until reaching the center

```python
def radial_path_to(self: HexPosition, center: HexPosition) -> list[HexPosition]:
    """Return path that goes through one hex per ring, moving toward center."""
    
    start_ring = self.distance(center)
    if start_ring == 0:
        return [self]
    
    path = [self]
    current = self
    
    # Move inward one ring at a time
    for ring_num in range(start_ring - 1, -1, -1):
        # Find closest hex in this ring
        next_hex = current.closest_in_ring(center, ring_num)
        path.append(next_hex)
        current = next_hex
    
    return path
```

---

## Chapter 6: Spiraling Outward (The Dance of Discovery)

Sometimes a bee needs to search the area around the Queen in an expanding spiral pattern. This is how they explore their territory efficiently.

```python
# Generate spiral from Queen outward to ring 2
queen = HexPosition(0, 0, 0)
spiral_cells = queen.spiral(2)

print(f"Spiral pattern contains {len(spiral_cells)} cells:")
print(f"  Ring 0: 1 cell")
print(f"  Ring 1: 6 cells") 
print(f"  Ring 2: 12 cells")
print(f"  Total: {len(spiral_cells)} cells")
# Output: Total: 19 cells
```

The spiral visits:
1. The center (ring 0)
2. All of ring 1 (6 cells)
3. All of ring 2 (12 cells)

This is perfect for "search all cells within distance N" operations!

---

## Chapter 7: Drawing Lines (The Messenger's Flight)

Beatrice needs to send a message to her sister, Bella, who is in a different part of the hive. The message should travel in a straight line through the honeycomb.

```python
# Beatrice's position
beatrice = HexPosition(0, 0, 0)

# Bella's position
bella = HexPosition(4, -2, -2)

# Find the straight-line path between them
flight_path = beatrice.line_to(bella)

print(f"Message travels through {len(flight_path)} cells:")
for i, pos in enumerate(flight_path):
    print(f"  Cell {i}: {pos}")
```

Output:
```
Message travels through 5 cells:
  Cell 0: HexPosition(0, 0, 0)
  Cell 1: HexPosition(1, 0, -1)
  Cell 2: HexPosition(2, -1, -1)
  Cell 3: HexPosition(3, -1, -2)
  Cell 4: HexPosition(4, -2, -2)
```

### Linear Interpolation in Cube Space

The `line_to` method uses linear interpolation (lerp) in cube coordinates:

```python
def line_to(self: HexPosition, other: HexPosition) -> list[HexPosition]:
    """Get line of hexes from self to other using linear interpolation."""
    n = self.distance(other)
    if n == 0:
        return [self]
    
    results = []
    for i in range(n + 1):
        t = i / n if n > 0 else 0
        # Lerp in cube coordinates
        q = self.q + (other.q - self.q) * t
        r = self.r + (other.r - self.r) * t
        s = self.s + (other.s - self.s) * t
        
        # Round to nearest hex
        results.append(HexPosition._cube_round(q, r, s))
    
    return results
```

The magic is in `_cube_round`—it takes fractional cube coordinates and finds the nearest valid hex position!

---

## Chapter 8: Rotation (The Waggle Dance)

Bees communicate through dance! Beatrice performs a waggle dance by rotating around the Queen's chamber.

### Rotating 60° Counter-Clockwise

```python
# A bee at the East position
bee = HexPosition(1, 0, -1)

# Rotate 60° counter-clockwise
rotated = bee.rotate_left()
print(f"After rotating left: {rotated}")
# Output: HexPosition(1, -1, 0) - now at NE position!
```

### Rotating 60° Clockwise  

```python
# Same bee rotates right
rotated = bee.rotate_right()
print(f"After rotating right: {rotated}")
# Output: HexPosition(0, 1, -1) - now at SE position!
```

### The Rotation Mathematics

```python
def rotate_left(self: HexPosition) -> HexPosition:
    """Rotate 60 degrees counter-clockwise around origin."""
    return HexPosition(-self.s, -self.q, -self.r)

def rotate_right(self: HexPosition) -> HexPosition:
    """Rotate 60 degrees clockwise around origin."""
    return HexPosition(-self.r, -self.s, -self.q)
```

This is a beautiful property of cube coordinates—rotation is just a cyclic permutation of the coordinates with sign changes!

---

## Chapter 9: Waypoints (The Foraging Route)

Beatrice has discovered multiple food sources and needs to visit them in order before returning home.

```python
# Starting position (outer hive)
start = HexPosition(5, 0, -5)

# Important locations to visit
flower_patch = HexPosition(3, -1, -2)
water_source = HexPosition(1, -1, 0)

# Final destination: the Queen
queen = HexPosition(0, 0, 0)

# Calculate complete route
route = start.path_through_waypoints(
    waypoints=[flower_patch, water_source],
    destination=queen
)

print(f"Beatrice's foraging route ({len(route)} stops):")
for i, pos in enumerate(route):
    print(f"  Stop {i}: {pos}")
```

This creates an optimized path that:
1. Goes from start → flower_patch (radially)
2. Goes from flower_patch → water_source (radially)
3. Goes from water_source → queen (radially)

---

## Chapter 10: Practical Applications

### Application 1: Finding All Neighbors

```python
def get_all_neighbors(position: HexPosition) -> list[HexPosition]:
    """Get the 6 hexes immediately surrounding a position."""
    directions = HexPosition.directions()
    return [position + direction for direction in directions]

# Example
cell = HexPosition(2, -1, -1)
neighbors = get_all_neighbors(cell)
print(f"Cell {cell} has {len(neighbors)} neighbors")
```

### Application 2: Field of View (What Can the Bee See?)

```python
def field_of_view(position: HexPosition, vision_range: int) -> list[HexPosition]:
    """Get all hexes within vision range."""
    return position.spiral(vision_range)

# Beatrice can see 2 cells in any direction
beatrice_pos = HexPosition(1, 0, -1)
visible_cells = field_of_view(beatrice_pos, 2)
print(f"Beatrice can see {len(visible_cells)} cells")
# Output: 19 cells (herself + rings 1 and 2 around her)
```

### Application 3: Area of Effect

```python
def honeycomb_explosion(center: HexPosition, radius: int) -> list[HexPosition]:
    """Get all hexes affected by an event centered at position."""
    affected = []
    for r in range(radius + 1):
        affected.extend(center.ring(r))
    return affected

# A scent marker affects 1 ring around it
marker = HexPosition(0, 0, 0)
affected = honeycomb_explosion(marker, 1)
print(f"Scent marker affects {len(affected)} cells")
# Output: 7 cells (center + 6 neighbors)
```

### Application 4: Border Detection

```python
def is_on_border(pos: HexPosition, territory_radius: int) -> bool:
    """Check if position is on the edge of the territory."""
    origin = HexPosition(0, 0, 0)
    return pos.distance(origin) == territory_radius

# Check if position is on the border of a radius-3 territory
edge_cell = HexPosition(3, 0, -3)
print(f"On border? {is_on_border(edge_cell, 3)}")
# Output: True
```

---

## Chapter 11: Advanced Concepts

### Converting to Pixel Coordinates

When drawing the hive on screen, we need to convert cube coordinates to x, y pixels:

```python
from primitives import MapCord

# Beatrice's cube position
beatrice = HexPosition(2, -1, -1)

# Convert to pixel coordinates for pointy-top hex
hex_radius = 30.0
center = MapCord(400, 300)  # Screen center

pixel_pos = beatrice.pixel(hex_radius, center)
print(f"Beatrice appears at pixel: ({pixel_pos.x:.1f}, {pixel_pos.y:.1f})")
```

The conversion formula for pointy-top hexagons:
```python
x = radius * (sqrt(3) * q + sqrt(3)/2 * r)
y = radius * (3/2 * r)
```

### Encoding and Decoding

Bees need to remember positions efficiently:

```python
# Encode position as string
beatrice = HexPosition(3, -1, -2)
encoded = beatrice.encode()
print(f"Encoded: {encoded}")
# Output: "3^-1^-2"

# Decode back to position
decoded = HexPosition.decode(encoded)
print(f"Decoded: {decoded}")
# Output: HexPosition(3, -1, -2)
```

---

## Chapter 12: Common Patterns and Idioms

### Pattern 1: Ring Walking

Visit every cell in a specific ring:

```python
queen = HexPosition(0, 0, 0)
ring_2_cells = queen.ring(2)

for cell in ring_2_cells:
    print(f"Inspecting cell: {cell}")
```

### Pattern 2: Range Query

Find all cells within a distance:

```python
center = HexPosition(0, 0, 0)
max_distance = 3

nearby = [pos for pos in center.spiral(max_distance)]
print(f"Found {len(nearby)} cells within distance {max_distance}")
```

### Pattern 3: Direction-Based Movement

Move N steps in a specific direction:

```python
start = HexPosition(0, 0, 0)
direction = HexPosition.directions()[0]  # East

# Move 5 steps East
position = start
for _ in range(5):
    position = position + direction

print(f"After 5 steps East: {position}")
# Output: HexPosition(5, 0, -5)
```

### Pattern 4: Manhattan Distance Alternative

Cube coordinates give you the true hexagonal distance, which is cleaner than Manhattan distance for hex grids:

```python
# These two cells look far apart in x,y coordinates
a = HexPosition(0, 0, 0)
b = HexPosition(3, -1, -2)

# But in hex space, they're only 3 steps apart
print(f"Hex distance: {a.distance(b)}")
# Output: 3
```

---

## Chapter 13: Why Cube Coordinates?

You might wonder: why use three coordinates when hexagons are 2D?

### Advantage 1: Symmetric and Elegant

All six directions are treated equally. Compare:

**Offset Coordinates** (row, col):
- Different formulas for even/odd rows
- Asymmetric neighbor calculations
- Harder to reason about

**Cube Coordinates** (q, r, s):
- Same formula works everywhere
- Symmetric in all six directions
- Intuitive distance and rotation

### Advantage 2: Simple Arithmetic

Moving in directions is just addition:
```python
new_pos = current_pos + direction
```

Distance is simple:
```python
distance = (|Δq| + |Δr| + |Δs|) / 2
```

### Advantage 3: The Constraint Helps

The constraint q + r + s = 0 means:
- You can always calculate the third coordinate
- Invalid positions are impossible to create
- Many algorithms become simpler

### Advantage 4: No Special Cases

With offset coordinates, edges and corners need special handling. With cube coordinates, the formulas work everywhere.

---

## Chapter 14: The Queen's Wisdom (Key Takeaways)

As Beatrice finally returns to the Queen's chamber with her important insights, the Queen shares the fundamental wisdom of hexagonal navigation:

### The Three Laws of the Honeycomb

1. **The Law of Balance**: q + r + s always equals 0
   - This keeps all positions valid
   - Third coordinate is always derivable from the other two

2. **The Law of Symmetry**: All six directions are equal
   - No direction is "special" or requires different handling
   - Rotation is just permutation with sign changes

3. **The Law of Rings**: Distance is measured in rings
   - Ring 0 is the center (1 cell)
   - Ring N contains 6N cells (for N > 0)
   - Distance is the ring number you're on

### Quick Reference

```python
# Create a position
pos = HexPosition(q=2, r=-1, s=-1)

# Get directions
directions = HexPosition.directions()  # Returns 6 directions: E, NE, NW, W, SW, SE

# Move
new_pos = pos + directions[0]  # Move East

# Distance
dist = pos.distance(other_pos)

# Rings
ring_cells = center.ring(radius)      # Cells in ring
all_cells = center.spiral(radius)     # All cells up to ring

# Paths
path = start.line_to(end)             # Straight line
path = start.radial_path_to(center)   # Inward spiral

# Rotation
rotated = pos.rotate_left()   # 60° counter-clockwise
rotated = pos.rotate_right()  # 60° clockwise

# Neighbor
neighbor = pos.neighbor(direction_index)  # 0-5 for six directions
```

---

## Epilogue: The Insight

Beatrice's important insight? She discovered that cube coordinates make the hive's hexagonal structure not just navigable, but *elegant*. Every bee can find their way home, every path can be calculated, and every dance can be understood—all because of three simple numbers that always sum to zero.

The Queen smiles. Her hive is not just a collection of hexagonal cells—it's a coordinate system, a graph, a geometric wonder. And every bee, by understanding their position in cube coordinates, can navigate it with mathematical precision.

---

## Appendix: Further Exploration

### Try These Exercises

1. **The Scout's Challenge**: Create a bee at position (4, -2, -2). Find all positions exactly 3 steps away.

2. **The Guard's Route**: Create a patrol route that visits all cells in ring 2, starting from the East and going clockwise.

3. **The Forager's Problem**: Find the shortest path from (-3, 2, 1) to (3, -2, -1) without going through the center.

4. **The Architect's Task**: Generate all cells in a hexagonal shape of radius 4, then count how many cells are on even vs odd rings.

### Additional Resources

- The `HexPosition` class in `primitives.py` implements all these concepts
- Check `HexGrid` for converting between grid indices and cube coordinates
- Look at `terrain.py` for practical applications in map generation

### The Mathematical Beauty

Cube coordinates reveal something profound: hexagonal grids aren't just 2D structures that happen to tile the plane. They're projections of a 3D cubic lattice onto a 2D plane where q + r + s = 0. This is why the math works so elegantly—we're not fighting the geometry, we're embracing it.

---

*"In the hive, every bee knows her place, every cell has its coordinates, and the Queen sits at the perfect center of a mathematical harmony."*

— End of Transmission —